In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import * #for window

spark = SparkSession\
.builder\
.master("yarn")\
.appName("window functions")\
.enableHiveSupport()\
.config("spark.sql.warehouse.dir","/user/itv009490/warehouse")\
.getOrCreate()

spark

In [2]:
df = spark.read.format("csv").\
option("inferSchema","true").\
option("header","true").\
load("/public/trendytech/datasets/windowdatamodified.csv")

In [3]:
df.show()

+--------------+-------+-----------+-------------+------------+
|       country|weeknum|numinvoices|totalquantity|invoicevalue|
+--------------+-------+-----------+-------------+------------+
|         Spain|     49|          1|           67|      174.72|
|       Germany|     48|         11|         1795|      1600.0|
|     Lithuania|     48|          3|          622|     1598.06|
|       Germany|     49|         12|         1852|      1800.0|
|       Bahrain|     51|          1|           54|      205.74|
|       Iceland|     49|          1|          319|      711.79|
|         India|     51|          5|           95|       300.0|
|     Australia|     50|          2|          133|      387.95|
|         Italy|     49|          1|           -2|       -17.0|
|         India|     49|          5|         1280|      3284.1|
|         Spain|     50|          2|          400|     1049.01|
|United Kingdom|     51|        200|        28782|    75103.46|
|        Norway|     49|          1|    

In [4]:
df.orderBy("country","invoicevalue").show()

+---------------+-------+-----------+-------------+------------+
|        country|weeknum|numinvoices|totalquantity|invoicevalue|
+---------------+-------+-----------+-------------+------------+
|      Australia|     49|          1|          214|       258.9|
|      Australia|     48|          1|          107|      358.25|
|      Australia|     50|          2|          133|      387.95|
|        Austria|     50|          2|            3|      257.04|
|        Bahrain|     51|          1|           54|      205.74|
|        Belgium|     50|          2|          285|      625.16|
|        Belgium|     48|          1|          528|       800.0|
|        Belgium|     51|          2|          942|       800.0|
|Channel Islands|     49|          1|           80|      363.53|
|         Cyprus|     50|          1|          917|     1590.82|
|        Denmark|     49|          1|          454|      1281.5|
|        Finland|     50|          1|         1254|       892.8|
|         France|     49|

In [5]:
# running total
my_window = Window.partitionBy("country").orderBy("weeknum").rowsBetween(Window.unboundedPreceding,Window.currentRow)

In [6]:
result = df.withColumn("runningTotal",sum("invoicevalue").over(my_window))
result.orderBy("country","weeknum").show()

+---------------+-------+-----------+-------------+------------+------------------+
|        country|weeknum|numinvoices|totalquantity|invoicevalue|      runningTotal|
+---------------+-------+-----------+-------------+------------+------------------+
|      Australia|     48|          1|          107|      358.25|            358.25|
|      Australia|     49|          1|          214|       258.9|            617.15|
|      Australia|     50|          2|          133|      387.95|1005.0999999999999|
|        Austria|     50|          2|            3|      257.04|            257.04|
|        Bahrain|     51|          1|           54|      205.74|            205.74|
|        Belgium|     48|          1|          528|       800.0|             800.0|
|        Belgium|     50|          2|          285|      625.16|1425.1599999999999|
|        Belgium|     51|          2|          942|       800.0|           2225.16|
|Channel Islands|     49|          1|           80|      363.53|            

In [7]:
my_window = Window.partitionBy("country").orderBy(desc("invoicevalue"))

In [8]:
result = df.withColumn("rank",rank().over(my_window))
result.show()

+-------+-------+-----------+-------------+------------+----+
|country|weeknum|numinvoices|totalquantity|invoicevalue|rank|
+-------+-------+-----------+-------------+------------+----+
| Sweden|     50|          3|         3714|      2646.3|   1|
|Germany|     49|         12|         1852|      1800.0|   1|
|Germany|     50|         15|         1973|      1800.0|   1|
|Germany|     48|         11|         1795|      1600.0|   3|
|Germany|     51|          5|         1103|      1600.0|   3|
| France|     50|          6|          529|      537.32|   1|
| France|     51|          5|          847|       500.0|   2|
| France|     49|          9|         2303|       500.0|   2|
| France|     48|          4|         1299|       500.0|   2|
|Belgium|     48|          1|          528|       800.0|   1|
|Belgium|     51|          2|          942|       800.0|   1|
|Belgium|     50|          2|          285|      625.16|   3|
|Finland|     50|          1|         1254|       892.8|   1|
|  India

In [9]:
result = df.withColumn("rank",dense_rank().over(my_window))
result.show()

+-------+-------+-----------+-------------+------------+----+
|country|weeknum|numinvoices|totalquantity|invoicevalue|rank|
+-------+-------+-----------+-------------+------------+----+
| Sweden|     50|          3|         3714|      2646.3|   1|
|Germany|     49|         12|         1852|      1800.0|   1|
|Germany|     50|         15|         1973|      1800.0|   1|
|Germany|     48|         11|         1795|      1600.0|   2|
|Germany|     51|          5|         1103|      1600.0|   2|
| France|     50|          6|          529|      537.32|   1|
| France|     51|          5|          847|       500.0|   2|
| France|     49|          9|         2303|       500.0|   2|
| France|     48|          4|         1299|       500.0|   2|
|Belgium|     48|          1|          528|       800.0|   1|
|Belgium|     51|          2|          942|       800.0|   1|
|Belgium|     50|          2|          285|      625.16|   2|
|Finland|     50|          1|         1254|       892.8|   1|
|  India

In [10]:
result = df.withColumn("rank",row_number().over(my_window))
result.show()

+-------+-------+-----------+-------------+------------+----+
|country|weeknum|numinvoices|totalquantity|invoicevalue|rank|
+-------+-------+-----------+-------------+------------+----+
| Sweden|     50|          3|         3714|      2646.3|   1|
|Germany|     49|         12|         1852|      1800.0|   1|
|Germany|     50|         15|         1973|      1800.0|   2|
|Germany|     48|         11|         1795|      1600.0|   3|
|Germany|     51|          5|         1103|      1600.0|   4|
| France|     50|          6|          529|      537.32|   1|
| France|     51|          5|          847|       500.0|   2|
| France|     49|          9|         2303|       500.0|   3|
| France|     48|          4|         1299|       500.0|   4|
|Belgium|     48|          1|          528|       800.0|   1|
|Belgium|     51|          2|          942|       800.0|   2|
|Belgium|     50|          2|          285|      625.16|   3|
|Finland|     50|          1|         1254|       892.8|   1|
|  India

In [14]:
result.select("*").where("rank ==1").drop("rank").show()

+---------------+-------+-----------+-------------+------------+
|        country|weeknum|numinvoices|totalquantity|invoicevalue|
+---------------+-------+-----------+-------------+------------+
|         Sweden|     50|          3|         3714|      2646.3|
|        Germany|     49|         12|         1852|      1800.0|
|         France|     50|          6|          529|      537.32|
|        Belgium|     48|          1|          528|       800.0|
|        Finland|     50|          1|         1254|       892.8|
|          India|     49|          5|         1280|      3284.1|
|          Italy|     48|          1|          164|       427.8|
|      Lithuania|     48|          3|          622|     1598.06|
|         Norway|     48|          1|         1852|     1919.14|
|          Spain|     50|          2|          400|     1049.01|
|        Denmark|     49|          1|          454|      1281.5|
|        Iceland|     49|          1|          319|      711.79|
|         Israel|     50|

In [12]:
my_window = Window.partitionBy("country").orderBy("weeknum")

In [20]:
result = df.withColumn("prev_week_value",lag("invoicevalue").over(my_window))
result.show()

+-------+-------+-----------+-------------+------------+---------------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|prev_week_value|
+-------+-------+-----------+-------------+------------+---------------+
| Sweden|     50|          3|         3714|      2646.3|           null|
|Germany|     48|         11|         1795|      1600.0|           null|
|Germany|     49|         12|         1852|      1800.0|         1600.0|
|Germany|     50|         15|         1973|      1800.0|         1800.0|
|Germany|     51|          5|         1103|      1600.0|         1800.0|
| France|     48|          4|         1299|       500.0|           null|
| France|     49|          9|         2303|       500.0|          500.0|
| France|     50|          6|          529|      537.32|          500.0|
| France|     51|          5|          847|       500.0|         537.32|
|Belgium|     48|          1|          528|       800.0|           null|
|Belgium|     50|          2|          285|      62

In [24]:
res2 = result.withColumn("prev_week_value",expr("case when prev_week_value is NULL then 0 else prev_week_value end as prev_week_value"))

In [30]:
res2.withColumn("change(%)",concat(ceil(expr("(invoicevalue-prev_week_value )/invoicevalue*100 ")),lit("%"))).show()

+-------+-------+-----------+-------------+------------+---------------+---------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|prev_week_value|change(%)|
+-------+-------+-----------+-------------+------------+---------------+---------+
| Sweden|     50|          3|         3714|      2646.3|            0.0|     100%|
|Germany|     48|         11|         1795|      1600.0|            0.0|     100%|
|Germany|     49|         12|         1852|      1800.0|         1600.0|      12%|
|Germany|     50|         15|         1973|      1800.0|         1800.0|       0%|
|Germany|     51|          5|         1103|      1600.0|         1800.0|     -12%|
| France|     48|          4|         1299|       500.0|            0.0|     100%|
| France|     49|          9|         2303|       500.0|          500.0|       0%|
| France|     50|          6|          529|      537.32|          500.0|       7%|
| France|     51|          5|          847|       500.0|         537.32|      -7%|
|Bel

In [31]:
my_wind=Window.partitionBy("country")

In [35]:
res3 = res2.withColumn("week_total",sum("invoicevalue").over(my_wind))
res3.show()

+-------+-------+-----------+-------------+------------+---------------+------------------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|prev_week_value|        week_total|
+-------+-------+-----------+-------------+------------+---------------+------------------+
| Sweden|     50|          3|         3714|      2646.3|            0.0|            2646.3|
|Germany|     48|         11|         1795|      1600.0|            0.0|            6800.0|
|Germany|     49|         12|         1852|      1800.0|         1600.0|            6800.0|
|Germany|     50|         15|         1973|      1800.0|         1800.0|            6800.0|
|Germany|     51|          5|         1103|      1600.0|         1800.0|            6800.0|
| France|     48|          4|         1299|       500.0|            0.0|2037.3200000000002|
| France|     49|          9|         2303|       500.0|          500.0|2037.3200000000002|
| France|     50|          6|          529|      537.32|          500.0|2037.320

In [39]:
res4 = res3.withColumn("week_contri",concat(ceil(expr("invoicevalue/week_total*100")),lit("%")))
res4.show()

+-------+-------+-----------+-------------+------------+---------------+------------------+-----------+
|country|weeknum|numinvoices|totalquantity|invoicevalue|prev_week_value|        week_total|week_contri|
+-------+-------+-----------+-------------+------------+---------------+------------------+-----------+
| Sweden|     50|          3|         3714|      2646.3|            0.0|            2646.3|       100%|
|Germany|     48|         11|         1795|      1600.0|            0.0|            6800.0|        24%|
|Germany|     49|         12|         1852|      1800.0|         1600.0|            6800.0|        27%|
|Germany|     50|         15|         1973|      1800.0|         1800.0|            6800.0|        27%|
|Germany|     51|          5|         1103|      1600.0|         1800.0|            6800.0|        24%|
| France|     48|          4|         1299|       500.0|            0.0|2037.3200000000002|        25%|
| France|     49|          9|         2303|       500.0|        